모듈 임포트

In [ ]:
from bs4 import BeautifulSoup 

import pandas as pd

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import time
import re
from tqdm import tqdm
from lxml import etree

크롬 드라이버 설치

In [ ]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

`빵집 이름`이 담긴 파일 가져오기
- `LISTLY` 사용하여 만든 엑셀 파일에서 이름만 따로 가져온다.
- 참고로 크롤링시 클릭이 먹히지 않기 때문에 해당 상호명을 검색했을 때, 원하는 지점 하나만 뜨도록 `빵집 이름`을 더 구체적으로 수정해준다. ex. 비파티세리->비파티세리 압구정
- 후에 만들어질 url 담을 컬럼도 만들어준다.

In [ ]:
df = pd.read_excel('서울시 마들렌 맛집] 리스트 (1018).xlsx') 
df['naver_map_url'] = ''
df = df.drop(columns=['Unnamed: 0'])
df

,검색어,방문자리뷰수,naver_map_url
0,멜로워 성수 더 플래그쉽,2386,
1,에뚜왈 마들렌 강남,1364,
2,르솔레이 마들렌,1306,
3,가또마들렌 암사점,1198,
4,메종엠오 방배로26길,1056,
5,츄이구이브레드,858,
6,마이스페이스 마들렌,790,
7,안녕과자점 마들렌,766,
8,마들마들 압구정,591,
9,버터풀앤크리멀러스 신사,566,


`빵집 이름` 이 담긴 컬럼의 이름을 "검색어"로 수정함.(엑셀)
1. 검색 url을 만들어 검색하고,
2. 검색된 플레이스의 페이지 url을 다시 가져온다.(검색 url의 가게코드 이용하는 원리로 추측됨.)
- 변수이름은 final_url. 
- 없는 경우 공백 

In [ ]:
for i, keyword in enumerate(df['검색어'].tolist()): 
    
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0]} 행", keyword) 
    
    try: 
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place' # 검색 url 만들기 
        driver.get(naver_map_search_url)
        time.sleep(5)

        cu = driver.current_url
        res_code = re.findall(r"place/(\d+)", cu)
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#' 
        
        print(final_url)
        df['naver_map_url'][i]=final_url 
        
    except IndexError: 
        df['naver_map_url'][i]= ''
        print('none')

이번에 찾을 키워드 : 0 / 10 행 멜로워 성수 더 플래그쉽
https://pcmap.place.naver.com/restaurant/365858525/review/visitor#
이번에 찾을 키워드 : 1 / 10 행 에뚜왈 마들렌 강남


C:\Users\YHS\AppData\Local\Temp\ipykernel_3720\2568378269.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_map_url'][i]=final_url


https://pcmap.place.naver.com/restaurant/996334906/review/visitor#
이번에 찾을 키워드 : 2 / 10 행 르솔레이 마들렌
https://pcmap.place.naver.com/restaurant/1915794004/review/visitor#
이번에 찾을 키워드 : 3 / 10 행 가또마들렌 암사점
https://pcmap.place.naver.com/restaurant/13157617/review/visitor#
이번에 찾을 키워드 : 4 / 10 행 메종엠오 방배로26길
https://pcmap.place.naver.com/restaurant/1152554768/review/visitor#
이번에 찾을 키워드 : 5 / 10 행 츄이구이브레드
https://pcmap.place.naver.com/restaurant/38282128/review/visitor#
이번에 찾을 키워드 : 6 / 10 행 마이스페이스 마들렌
https://pcmap.place.naver.com/restaurant/1906013465/review/visitor#
이번에 찾을 키워드 : 7 / 10 행 안녕과자점 마들렌
https://pcmap.place.naver.com/restaurant/1709687194/review/visitor#
이번에 찾을 키워드 : 8 / 10 행 마들마들 압구정
https://pcmap.place.naver.com/restaurant/1932943275/review/visitor#
이번에 찾을 키워드 : 9 / 10 행 버터풀앤크리멀러스 신사
none


C:\Users\YHS\AppData\Local\Temp\ipykernel_3720\2568378269.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_map_url'][i]= ''


### 각 매장별 리뷰 데이터프레임 만들기
리뷰 리스트 만들어서 데이터프레임으로 변환 후,
상호명 리스트(name_list)를 컬럼으로 지정함.
- 방법은 https://jenn1won.tistory.com/15 와 비슷하나,
- 필요하지 않은 데이터(작성자, 사진 등)은 코드에서 제거함.
- By , lxml 모듈에서 오류가 나서 따로 설치를 진행함.
- 셀렉터 값도 내 컴퓨터와 달라 수정해줌.  
- 더보기 버튼을 20번만 눌러서 약 200개 리뷰 수집
- 리뷰수가 다르면 데이터프레임 생성하는 데 오류 생겨서 리뷰수 200개로 잘라서 넣음.

In [ ]:
df_result=pd.DataFrame()

for i in range(len(df)): 
    review_list= []
    
    print('======================================================')
    
    # url 접속
    driver.get(df['naver_map_url'][i]) 
    thisurl = df['naver_map_url'][i]
    time.sleep(2) 
    
    # 더보기버튼 25번 클릭(한 번당 약 10개 뽑힌다. 넉넉히 25)
    num = 0
    while num < 25:
        try: 
            time.sleep(1) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(3) 
            
            driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.lfH3O > a').send_keys(Keys.ENTER) 
            time.sleep(3) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END) 
            time.sleep(1) 
            num += 1
        except : 'none'

    # 파싱
    html = driver.page_source 
    soup = BeautifulSoup(html, 'lxml') 
    time.sleep(1) 
    
    # 식당 이름, 구분, 총 개수 출력
    restaurant_name = df['검색어'][i]
    print('식당 이름 : '+restaurant_name)

    
    try: 
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'DJJvD'})[0].text 
    
    except: 
        restaurant_classificaton = 'none'
    
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try: 
        one_review = soup.find_all('div', attrs = {'class':'ZZ4OK IwhtZ'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수 
        print('리뷰 총 개수 : '+str(review_num)) 
        
        # 목표치인 리뷰 200개씩만 모으기
        for i in range(200):
            
            try: 
                review_content = one_review[i].find('span', attrs = {'class':'zPfVt'}).text
                review_list.append(review_content)
            except:
                review_content = "" 
            print('리뷰 내용 : '+review_content) 
            
      
    except NoSuchElementException: 
        none_review = "네이버 리뷰 없음" 
        print(none_review)
        review_num = 0 
        
    # 데이터 프레임 만들기
    df_result[f"{restaurant_name}"] = review_list.copy()

식당 이름 : 멜로워 성수 더 플래그쉽
식당 구분 : 카페,디저트
----------------------------------------------
리뷰 총 개수 : 199
리뷰 내용 : 매장이 넓고 힙해요~ 그리고 디저트 종류가 엄청 다양해요! 저희가 고른 건 생각만큼 엄청 맛있진 않았어서 조금 아쉽지만 테라스 자리에서 대화하기 
좋았어요ㅎ 멜로워 스파클링 음료가 맛있습니다!
리뷰 내용 : 디저트가 다양해서 선택장애인 저는 고르기가 힘들었어욤ㅜㅋㅋㅋ 피스타치오크림 올라가있는거 맛있었습니당ㅎㅎ 크로와상위 머랭올라간듯한 아이는 생각보다 평범! 다음엔 또 다른 디저트 먹어보고싶어요~ 화장실이 외부에도 있다지만 2층에는 남녀공용 1칸뿐이라 몰리면 기다림이 생기네요;;참고하시길
리뷰 내용 : 성수에서 갈 수 있는 규모 큰 카페 중 하나이지 않을까 싶어요. 그래도 주말에 방문하면 자리가 없어서 못 앉으실 수 있어요. 옥상 테라스와 야장도 자리가 있어서 바깥 구경하면서 즐기시는 것도 가능! 먹기는 다소 불편하지만 로그번이 맛있어요~
리뷰 내용 : 저번에 왔을때 보다 디저트가 다채로워졌어요!
밤 모양 신기하고 귀여어서 사봤는데 퀸아망안에 밤잼 가득한 맛이에요! 많이 달줄 알았는데 적당히 달고 맛있어요 식빵 안에 찐덕한 우유크림 걍 젖소 모양인거고초코 맛은 아님 
리뷰 내용 : 올 때마다 늘 인기가 많고 자리가 없습니다 😂 레몬 케이크는 처음 먹어보았는데 아주 새콤하고 재료를 아끼지 않은 좋은 맛이 났습니다. 스콘도 늘 맛있어요! 인테리어도 성수 감성이 제대로 살아 있어 좋습니다 👍
리뷰 내용 : 아무 생각없이 들어가봤는데 생각보다 좋더라고요. 빵, 디저트 종류 모두 특이하고 다 하나씩 먹고 싶어지네요🥰 역시 핫플이라 사람많고 정신이 없어 
보이지만 자리가 꽤 많은 편이라 앉을 수 있엇어요😁 재방문하고 싶은 곳이에용
리뷰 내용 : 디저트로 캐러멜얼그레이, 딸기에 빠진 소, 초콜릿 로그번 먹엇는데 전부다 달긴하지만 갠적으로 초콜릿 로그번이 젤 맛있었습니다 안에 크림들어가잇으

In [ ]:
df_result.to_excel('리뷰모음.xlsx')